# Korean unsmile

In [16]:
import gradio as gr
import torch
import tensorflow
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

In [4]:
# !pip3 install torch

In [6]:
labels = [
    "여성/가족",
    "남성",
    "성소수자",
    "인종/국적",
    "연령",
    "지역",
    "종교",
    "기타 혐오",
    "악플/욕설",
    "clean",
]

## 모델 가져오기

In [17]:
model = AutoModelForSequenceClassification.from_pretrained("smilegate-ai/kor_unsmile")
tokenizer = AutoTokenizer.from_pretrained("smilegate-ai/kor_unsmile")

ImportError: 
AutoModelForSequenceClassification requires the PyTorch library but it was not found in your environment.
However, we were able to find a TensorFlow installation. TensorFlow classes begin
with "TF", but are otherwise identically named to our PyTorch classes. This
means that the TF equivalent of the class you tried to import would be "TFAutoModelForSequenceClassification".
If you want to use TensorFlow, please use TF classes instead!

If you really do want to use PyTorch please go to
https://pytorch.org/get-started/locally/ and follow the instructions that
match your environment.


## 악플 감지 함수 구하기

In [18]:
## 중요중요 ##
def checkHateSpeech(text):
    with torch.no_grad(): # model의 gradient 계산 비활성화
        inputs = tokenizer(text, return_tensors='pt') # return_tensors : pytorch
        # 변환된 입력(inputs) 모델에 넣어 결과를 도출
        outputs = model(**inputs)
        
        logits = outputs['logits'] # logits : 승비

        # 원시 예측값을 확률로 변환한다. (sigmoid : 0 ~ 1 사이)
        probs = torch.sigmoid(logits)
        # 0 ~ 1 사이에 나오는 확률값 >> labels과 매칭 >> dict() 형태로 반환
        probs_by_labels = {labels[i] : float(probs[0][i]) for i in range(len(labels))}

        return probs_by_labels

## Gradio 화면 그리기

In [ ]:
iface = \
gr.Interface(
    fn=checkHateSpeech,
    inputs=gr.Textbox(label='Input Text'),
    outputs=gr.Label(label='혐오 발언 분석', num_top_classes=5),
    examples=[
        '사랑해요. 증오해요.'
    ],
)

iface.launch(share=True)